In [1]:
# 1.

# Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

# InvoiceNo — номер транзакции
# StockCode — код товара
# Description — описание товара
# Quantity — количество единиц товара, добавленных в заказ
# InvoiceDate — дата транзакции 
# UnitPrice — цена за единицу товара
# CustomerID — id клиента
# Country — страна, где проживает клиент

# В данной задаче ссылка для считывания данных лежит в переменной path_to_file.
# Импортируйте пандас и прочитайте данные с кодировкой ISO-8859-1.
# Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

path_to_file = '5_data.csv.zip'
import pandas as pd
retail_original = pd.read_csv(path_to_file, compression = 'zip', parse_dates = ['InvoiceDate'], encoding = 'ISO-8859-1')
retail_columns = retail_original.columns

In [3]:
retail_original.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [4]:
# 2.

# Проверьте, встречаются ли в данных повторяющиеся наблюдения,и в качестве ответа укажите их количество.
# Если они есть, то удалите их из retail.

retail = retail_original.drop_duplicates()
repeating_data = retail_original.shape[0] - retail.shape[0]
print(repeating_data)

5268


In [5]:
# 3.

# Данные содержат в себе записи как об успешных транзакциях, так и об отмененных.
# Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
# Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

canceled_orders = retail.InvoiceNo.str.startswith('C').value_counts()
canceled_orders

False    527390
True       9251
Name: InvoiceNo, dtype: int64

In [6]:
# 4.

# Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0.
# В качестве ответа укажите число оставшихся строк.

retail.query('Quantity > 0').shape[0]

526054

In [43]:
# 5.

# Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany).
# Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.
# Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).
# Отфильтрованные данные записаны в retail. Идентификатор заказа – InvoiceNo.
# Для каждого заказа в данных может встречаться более 1 строки.

germany_customer = retail.query('Country == "Germany"') \
    .groupby(['CustomerID'], as_index = False) \
    .agg({'InvoiceNo': 'nunique'})

germany_customer_q08 = germany_customer.InvoiceNo.quantile(q = 0.8) 
germany_top = germany_customer.query('InvoiceNo > @germany_customer_q08').CustomerID

In [15]:
retail_original.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [47]:
# 6.

# Используя объект с id пользователей (germany_top), полученный на предыдущем шаге,
# отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам.
# Результирующий датафрейм запишите в top_retail_germany.
# Датафрейм записан в retail, нужные id пользователей – в germany_top

top_retail_germany = retail.query('CustomerID in @germany_top')
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,2011-12-09 11:59:00,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,2011-12-09 11:59:00,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,2011-12-09 11:59:00,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,2011-12-09 11:59:00,4.95,12662.0,Germany


In [95]:
# 7.

# Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
# Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

top_retail_germany \
    .groupby(['StockCode'], as_index = False) \
    .agg({'InvoiceNo' : 'count'}) \
    .sort_values('InvoiceNo', ascending = False)

,StockCode,InvoiceNo
1167,POST,219
409,22326,62
452,22423,55
411,22328,45
474,22467,37
...,...,...
743,23008,1
739,22999,1
44,20717,1
333,22119,1


In [85]:
# 8.

# Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.
# Отфильтрованные данные записаны в retail.

retail['Revenue'] = retail.Quantity * retail.UnitPrice
retail

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


In [90]:
# 9.

# Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа.
# В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

TotalRevenue = retail.groupby(['InvoiceNo'], as_index = False) \
    .agg({'Revenue': 'sum'}) \
    .sort_values('Revenue', ascending = False) \
    .head(5) \
    .InvoiceNo
TotalRevenue

22025    581483
2303     541431
18776    574941
19468    576365
9741     556444
Name: InvoiceNo, dtype: object